In [1]:
import pyodbc
import os
from IPython.core.display import display
from dotenv import load_dotenv
import pandas as pd

Connecting to SQL Server

In [3]:
load_dotenv('credentials.env')

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+os.environ.get('server')+';'
    'DATABASE='+os.environ.get('database')+';'
    'UID='+ os.environ.get('uid') +';'
    'PWD='+ os.environ.get('pwd')+';')

1. Retrieve the products (and their respective storage locations) that are running low (under 50 units).

In [5]:
low_inventory_df = pd.read_sql("SELECT p.Name AS Product, "
                                      "l.Name AS Location, "
                                      "SUM(prod_inv.Quantity) AS TotalQuantity "
                                      "FROM Production.ProductInventory prod_inv "
                                      "INNER JOIN Production.[Location] l "
                                      "ON prod_inv.LocationID = l.LocationID "
                                      "INNER JOIN Production.Product p "
                                      "ON prod_inv.ProductID = p.ProductID "
                                      "GROUP BY l.Name, p.Name "
                                      "HAVING SUM(prod_inv.Quantity) < 50 "
                                      "ORDER BY l.Name, p.Name;", conn)

display(low_inventory_df)

,Product,Location,TotalQuantity
0,"Mountain-500 Black, 52",Final Assembly,30
1,"Mountain-500 Silver, 44",Final Assembly,30
2,"Road-250 Black, 48",Final Assembly,49
3,"Road-450 Red, 52",Final Assembly,49
4,"Road-650 Black, 60",Final Assembly,49
5,"Road-750 Black, 44",Final Assembly,30
6,"Touring-1000 Yellow, 54",Final Assembly,40
7,"Touring-2000 Blue, 60",Final Assembly,49
8,"Touring-3000 Blue, 54",Final Assembly,35
9,"Touring-3000 Yellow, 54",Final Assembly,36


2. Retrieve all products (Name and Model) and their inventory locations where safety stock level falls below or equal to the number of available units.

In [6]:
products_to_order_df = pd.read_sql('SELECT l.Name AS Location, '
                 'p.Name AS ProductName, '
                 'pm.Name AS Model, '
                 'p.SafetyStockLevel, '
                 'p.ReorderPoint, '
                 'TotalAvailableQty, '
                 'p.ReorderPoint - TotalAvailableQty AS UnitsToReorder '
                 'FROM Production.Product p '
                 'INNER JOIN (SELECT ProductID, '
                 'LocationID, '
                 'SUM(Quantity) OVER(PARTITION BY ProductID) AS TotalAvailableQty '
                 'FROM Production.ProductInventory) prod_inv '
                 'ON prod_inv.ProductID = p.ProductID '
                 'INNER JOIN Production.ProductModel pm '
                 'ON pm.ProductModelID = p.ProductModelID '
                 'INNER JOIN Production.[Location] l '
                 'ON prod_inv.LocationID = l.LocationID '
                 'WHERE TotalAvailableQty <= ReorderPoint '
                                   'ORDER BY l.Name;', conn)

display(products_to_order_df)

,Location,ProductName,Model,SafetyStockLevel,ReorderPoint,TotalAvailableQty,UnitsToReorder
0,Final Assembly,HL Mountain Seat/Saddle,HL Mountain Seat/Saddle 2,500,375,355,20
1,Final Assembly,"Touring-1000 Yellow, 54",Touring-1000,100,75,75,0
2,Finished Goods Storage,"Touring-1000 Yellow, 54",Touring-1000,100,75,75,0
3,Finished Goods Storage,"Women's Tights, M",Women's Tights,4,3,0,3
4,Finished Goods Storage,"Half-Finger Gloves, M",Half-Finger Gloves,4,3,0,3
5,Finished Goods Storage,Hitch Rack - 4-Bike,Hitch Rack - 4-Bike,4,3,0,3
6,Finished Goods Storage,"Short-Sleeve Classic Jersey, M",Short-Sleeve Classic Jersey,4,3,0,3
7,Miscellaneous Storage,HL Mountain Seat/Saddle,HL Mountain Seat/Saddle 2,500,375,355,20
8,Subassembly,HL Mountain Seat/Saddle,HL Mountain Seat/Saddle 2,500,375,355,20


3. Retrieve all products (ProductID, Name, Model) that do not have any orders.

In [5]:
orders_check_df = pd.read_sql("SELECT p.Name AS ProductName, "
                                    "pm.Name AS Model, "
                                    "SUM(pv.OnOrderQty) AS TotalOrderedQty "
                                    "FROM Production.Product p "
                                    "INNER JOIN Production.ProductModel pm "
                                    "ON pm.ProductModelID = p.ProductModelID "
                                    "INNER JOIN Purchasing.ProductVendor pv "
                                    "ON pv.ProductID = p.ProductID "
                                    "GROUP BY p.Name, pm.Name "
                                    "HAVING SUM(pv.OnOrderQty) IS NULL "
                                    "ORDER BY p.Name;", conn)

display(orders_check_df)



,ProductName,Model,TotalOrderedQty
0,Front Brakes,Front Brakes,None
1,HL Mountain Tire,HL Mountain Tire,None
2,HL Road Pedal,HL Road Pedal,None
3,HL Road Tire,HL Road Tire,None
4,HL Touring Seat/Saddle,HL Touring Seat/Saddle,None
5,LL Mountain Seat/Saddle,LL Mountain Seat/Saddle 2,None
6,LL Mountain Tire,LL Mountain Tire,None
7,LL Road Seat/Saddle,LL Road Seat/Saddle 1,None
8,LL Road Tire,LL Road Tire,None
9,LL Touring Seat/Saddle,LL Touring Seat/Saddle,None
